# Voluntarios Aprendo Contigo

In [78]:
import win32com.client
import pandas as pd
import numpy  as np
import re

from datetime import datetime, timedelta

## I. Limpiar texto de la lista

In [79]:
def limpiar_texto(texto):
    texto = texto.replace('\r', ' ').replace('\n', ' ')
    texto = re.sub(r'[^\x00-\x7F]+',' ', texto)
    return texto

# II. Clasificar datos del voluntario

In [80]:
def extraer_info_voluntario(cuerpo):
    
    # 01. Definir campos_correo
    campos_correo = ['Nombres y Apellidos', 'Nombres y apellidos', 'Telfono', 'Email',
                     'Edad', 'Por qu quieres participar como voluntario?', 'Direccin',
                     'Distrito de residencia', 'Universidad', 'Carrera', 'Ciclo', 'Profesin',
                     'Ocupacin', 'Hospital', 'Sede', 'Dia de voluntariado', 'Turno', 
                     'Acept acatar el "Reglamento del Voluntariado" de Aprendo Contigo',
                     'Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo', 
                     'Deseo recibir informacin del programa',
                     'Desea recibir informacin del programa',
                     'Fecha']
    
    # 02. Inicializar diccionario: valor inicial None
    info_voluntario = {campo: None for campo in campos_correo}

    # 03. Limpiar cuerpo del correo
    cuerpo_limpio = limpiar_texto(cuerpo)

    # 04. Dividir el contenido por cada campo
    for campo in campos_correo:
        
        # 05. Encontrar campo_correo en cuerpo
        # - rf'{campo}: busca el nombre del campo seguido de dos puntos.
        # - (.*?) captura cualquier cosa después de los dos puntos hasta que encuentre el siguiente campo o el final del texto.
        # - (?=\w+:|$) asegura que la captura termine antes del siguiente campo (indicado por una palabra seguida de dos puntos) o el final del texto ($).
        #campo_encontrado = re.search(rf'{campo}:(.*?)(?=\w+:|$)', cuerpo_limpio)
        campo_encontrado = re.search(rf'{campo}[:\s]*(.*?)(?=\w+:|$)', cuerpo_limpio, re.DOTALL)
        
        # 06. Extrae el contenido y lo asigna al campo correspondiente en el diccionario
        if campo_encontrado:
            info_voluntario[campo] = campo_encontrado.group(1).strip()

    return info_voluntario

In [81]:
def extraer_info_turno(cuerpo):
    # Patrón para buscar el turno de mañana y tarde con sus respectivos horarios
    patron_turno_manana = r'Turno[:\s]*Ma[aá]n[aa]s?[:\s]*([^\n\r]*)'
    patron_turno_tarde = r'Turno[:\s]*Tard[ee]s?[:\s]*([^\n\r]*)'
    
    # Función auxiliar para limpiar el texto capturado
    def limpiar_texto(cadena):
        if cadena:
            return cadena.strip()
        return None
    
    # Buscar el turno de la mañana
    turno_manana_encontrado = re.search(patron_turno_manana, cuerpo, re.IGNORECASE | re.DOTALL)
    if turno_manana_encontrado:
        turno_manana = limpiar_texto(turno_manana_encontrado.group(1))
        if turno_manana:
            return f"Mañana: {turno_manana}"
    
    # Buscar el turno de la tarde
    turno_tarde_encontrado = re.search(patron_turno_tarde, cuerpo, re.IGNORECASE | re.DOTALL)
    if turno_tarde_encontrado:
        turno_tarde = limpiar_texto(turno_tarde_encontrado.group(1))
        if turno_tarde:
            return f"Tarde: {turno_tarde}"
    
    return None

# III. Extrar información de los correos

In [82]:
L_correos      = []
cuerpo_correos = []

# 01. Conectar a Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
print('-- 01. Conectar a Outlook')

# 02. Identificar cuenta Aprendo Contigo
cuenta_aprendo = None

for cuenta in outlook.Folders:
    if cuenta.Name == 'psicologia@aprendocontigo.org':
        cuenta_aprendo = cuenta
        print('-- 02. Cuenta Aprendo encontrada:', cuenta_aprendo)
        break

# 03. A) No encontro cuenta: No pasa nada
if cuenta_aprendo is None:
    print('-- !! No se encontro la cuenta Aprendo Contigo')

# 03. B) Encontro cuenta: Procesar correos
else:
    print('-- 03. Entrar en cuenta')
    try:
        # 04. Entrar a la bandeja de entrada
        bandeja_entrada = cuenta_aprendo.Folders['Bandeja de entrada']
        print('-- 04. Entrar a bandeja de entrada')

        # 05. Leer correos: más reciente primero
        correos = bandeja_entrada.Items
        correos.Sort("[ReceivedTime]", True)
        print('-- 05. Correos encontrados:', correos.count)

        # 06. Filtro: Voluntarios
        filtro = "@SQL=\"urn:schemas:httpmail:subject\" LIKE '%WEB-FORMULARIO VOLUNTARIADO%'"
        correos_voluntarios = correos.Restrict(filtro)
        print('-- 06. Correos de voluntarios:', correos_voluntarios.count)

        contador = 1

        # 07. Procesar información de los correos
        print('-- 07. Correos encontrados')
        
        for correo in correos_voluntarios:
            print()
            asunto      = correo.Subject
            f_recepcion = correo.ReceivedTime
            cuerpo      = correo.Body

            # Limpiar cuerpo de caracteres no ASCII
            # cuerpo = re.sub(r'[^\x00-\x7F]+',' ', cuerpo)
            cuerpo = correo.Body.encode('ascii', 'ignore').decode()

            print(contador, '-- Asunto correo:', asunto)
            
            L_correos.append(asunto)
            cuerpo_correos.append(cuerpo)

            print('Asunto:', asunto)
            print('Emisor:', correo.SenderName)
            print('Recibido:', f_recepcion)
            print('Cuerpo:', cuerpo) # Imprimir 100 primeros caracteres

            contador += 1

    except Exception as e:
        print('-- !! Error al entrar a bandeja de entrada:', e)

-- 01. Conectar a Outlook
-- 02. Cuenta Aprendo encontrada: psicologia@aprendocontigo.org
-- 03. Entrar en cuenta
-- 04. Entrar a bandeja de entrada
-- 05. Correos encontrados: 4935
-- 06. Correos de voluntarios: 876
-- 07. Correos encontrados

1 -- Asunto correo: WEB-FORMULARIO VOLUNTARIADO Emily Soto Henostroza
Asunto: WEB-FORMULARIO VOLUNTARIADO Emily Soto Henostroza
Emisor: aprendocontigo.org
Recibido: 2024-07-16 16:26:45+00:00
Cuerpo: Datos Personales:
Nombres y Apellidos: Emily Soto Henostroza
Telfono: 929781199
Email: emilyvivianasoto@gmail.com
Edad: 20
Distrito de residencia: Comas
Universidad: Universidad privada del norte
Carrera: Psicologa
Ciclo: 05
Sede: Sede INSN (Brea)
Dia de voluntariado: Lunes Mircoles Viernes
Turno: Maanas: 8:30 am a 1:00pm
Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo: on
Deseo recibir informacin del programa: Si
---

Fecha: julio 16, 2024
Hora: 9:26 pm
URL de la pgina: https://aprendocontigo.org/voluntariado/?fbclid=PAZXh0bgNhZW0CMT

## 1. Crear Dataframe con la información de los voluntarios

In [83]:
# 08. Procesar información de los voluntarios
voluntarios_info = [extraer_info_voluntario(cuerpo) for cuerpo in cuerpo_correos]
df_voluntarios   = pd.DataFrame(voluntarios_info)
df_voluntarios.head()

,Nombres y Apellidos,Nombres y apellidos,Telfono,Email,Edad,Por qu quieres participar como voluntario?,Direccin,Distrito de residencia,Universidad,Carrera,...,Ocupacin,Hospital,Sede,Dia de voluntariado,Turno,"Acept acatar el ""Reglamento del Voluntariado"" de Aprendo Contigo",Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo,Deseo recibir informacin del programa,Desea recibir informacin del programa,Fecha
0,Emily Soto Henostroza,None,929781199,emilyvivianasoto@gmail.com,20 Distrito de,None,None,Comas,Universidad privada del norte,Psicologa,...,None,None,Sede INSN (Brea) Dia de,Lunes Mircoles Viernes,,None,on Deseo recibir informacin del,Si ---,None,"julio 16, 2024"
1,Emily Soto Henostroza,None,929781199,emilyvivianasoto@gmail.com,20 Distrito de,None,None,Comas,Universidad privada del norte,Psicologa,...,None,None,Sede INSN (Brea) Dia de,Lunes Viernes Mircoles,,None,on Deseo recibir informacin del,Si ---,None,"julio 16, 2024"
2,Emily Soto Henostroza,None,929781199,emilyvivianasoto@gmail.com,20 Distrito de,None,None,Comas,Universidad privada del norte,Psicologa,...,None,None,Sede INSN (Brea) Dia de,Lunes Mircoles Viernes,,None,on Deseo recibir informacin del,Si ---,None,"julio 16, 2024"
3,Shalom Milagro Velsquez Daz,None,936858793,1412shalom.vd10@gmail.com,20 Distrito de,None,None,Santa Anita,Universidad Femenina del Sagrado Corazn,Psicologa,...,None,None,Sede INEN (Surquillo) Dia de,Martes Jueves,,None,on Deseo recibir informacin del,Si ---,None,"julio 16, 2024"
4,Abraham Elas Aguilar Hilario,None,928517058,abrahamhoppers@gmail.com,29 Distrito de,None,None,San Juan de Lurigancho,Universidad Privada del Norte,Psicologa,...,None,None,Sede INEN (Surquillo) Dia de,Lunes Martes Mircoles Viernes Sbado,,None,on Deseo recibir informacin del,Si ---,None,"julio 16, 2024"


In [84]:
turnos_extraidos = [extraer_info_turno(cuerpo) for cuerpo in cuerpo_correos]

df_turno = pd.DataFrame(turnos_extraidos)
df_turno

,0
0,Mañana: 8:30 am a 1:00pm
1,Mañana: 8:30 am a 1:00pm
2,Mañana: 8:30 am a 1:00pm
3,Tarde: 2:30 pm a 6:00pm
4,Mañana: 8:30 am a 1:00pm
...,...
871,Mañana: //
872,Mañana: //
873,Tarde: //
874,Tarde: //


In [85]:
df_voluntarios.columns

Index(['Nombres y Apellidos', 'Nombres y apellidos', 'Telfono', 'Email',
       'Edad', 'Por qu quieres participar como voluntario?', 'Direccin',
       'Distrito de residencia', 'Universidad', 'Carrera', 'Ciclo', 'Profesin',
       'Ocupacin', 'Hospital', 'Sede', 'Dia de voluntariado', 'Turno',
       'Acept acatar el "Reglamento del Voluntariado" de Aprendo Contigo',
       'Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo',
       'Deseo recibir informacin del programa',
       'Desea recibir informacin del programa', 'Fecha'],
      dtype='object')

## 2. Limpieza de datos

In [86]:
D_Datos = {}

def tipos_datos(campo):
    for i in df_voluntarios[campo]:
        tipo_dato = type(i)
        if tipo_dato in D_Datos:
            D_Datos[tipo_dato] += 1
        else:
            D_Datos[tipo_dato] = 1
    
    return D_Datos

In [87]:
def limpiar_num(campo):
    if isinstance(campo, str):
        solo_digitos = re.sub(r'\D', '', campo)
        return int(solo_digitos) if solo_digitos else None

In [88]:
def limpiar_texto(campo):
    if isinstance(campo, str):
        solo_texto = re.sub(r'[^\w\s]', '', campo).strip()
        return solo_texto
    else:
        return campo

### 2.1. Nombres y apellidos

In [89]:
df_voluntarios['Nombres y Apellidos'] = df_voluntarios['Nombres y apellidos'].fillna(df_voluntarios['Nombres y Apellidos'])

In [91]:
df_voluntarios.drop(columns=['Nombres y apellidos'], inplace=True)

In [93]:
df_voluntarios['Nombres y Apellidos'] = df_voluntarios['Nombres y Apellidos'].apply(limpiar_texto)

In [95]:
df_voluntarios['Nombres y Apellidos'] = df_voluntarios['Nombres y Apellidos'].str.title()

### Eliminar filas cuyos "Nombres y apellidos" estén vacíos

In [97]:
df_voluntarios = df_voluntarios.dropna(subset=['Nombres y Apellidos'])

In [99]:
df_voluntarios = df_voluntarios[df_voluntarios['Nombres y Apellidos'].str.strip() != '']

### 2.2. Teléfono

In [101]:
def limpiar_tel(campo):
    if isinstance(campo, str):
        solo_digitos = re.sub(r'\D', '', campo)
        return solo_digitos
    else:
        return campo 

In [102]:
df_voluntarios['Telfono'] = df_voluntarios['Telfono'].apply(limpiar_tel)

### 2.3. Email

In [103]:
def limpiar_email(campo):
    if isinstance(campo, str):
        # Eliminar caracteres no deseados
        solo_texto = re.sub(r'[^\w\s@.]', '', campo).strip()
        
        # Verificar si contiene '@' y '.'
        if '@' in solo_texto and '.' in solo_texto:
            # Cortar en el primer '.com', '.net', '.org', etc.
            extensiones_validas = ['.com', '.pe']
            for extension in extensiones_validas:
                if extension in solo_texto:
                    solo_texto = solo_texto.split(extension)[0] + extension
                    break
            return solo_texto
    return None

In [104]:
df_voluntarios['Email'] = df_voluntarios['Email'].apply(limpiar_email)

### 2.4. Edad

In [105]:
df_voluntarios['Edad'] = df_voluntarios['Edad'].apply(limpiar_num)

### 2.5. Motivo: ¿Por qué desea participar como voluntario?

In [106]:
df_voluntarios['Motivo'] = df_voluntarios['Por qu quieres participar como voluntario?'].apply(limpiar_texto)

In [107]:
df_voluntarios['Motivo'] = df_voluntarios['Motivo'].str.replace(r'\t\s*\tActualmente trabajas', '', regex=True).str.strip()

In [108]:
df_voluntarios.drop(columns=['Por qu quieres participar como voluntario?'], inplace=True)

### 2.6. Dirección

In [109]:
df_voluntarios['Direccin'] = df_voluntarios['Direccin'].str.extract(r'^(.*?)//')[0].str.strip()

### 2.7. Distrito de residencia

In [110]:
df_voluntarios['Distrito de residencia'] = df_voluntarios['Distrito de residencia'].str.title()

### 2.8. Universidad

In [112]:
df_voluntarios['Universidad'] = df_voluntarios['Universidad'].str.title()

### 2.9. Carrera

In [113]:
df_voluntarios['Carrera'] = df_voluntarios['Carrera'].str.title()

### 2.10. Ciclo

In [114]:
df_voluntarios['Ciclo'] = df_voluntarios['Ciclo'].apply(limpiar_num)

### 2.11. Profesión

In [115]:
df_voluntarios['Profesión'] = df_voluntarios['Profesin'].str.extract(r'^(.*?)//')[0].str.strip()

In [116]:
df_voluntarios.drop(columns=['Profesin'], inplace=True)

### 2.12. Ocupación

In [117]:
df_voluntarios['Ocupación'] = df_voluntarios['Ocupacin'].str.extract(r'^(.*?)//')[0].str.strip()

In [118]:
df_voluntarios.drop(columns=['Ocupacin'], inplace=True)

### 2.13. Hospital

In [119]:
df_voluntarios['Hospital'] = df_voluntarios['Hospital'].str.extract(r'^(.*?)//')[0].str.strip()

### 2.14. Sede

In [120]:
df_voluntarios['Sede'] = df_voluntarios['Sede'].str.extract(r'^(.*?\))')[0].str.strip()

In [121]:
df_voluntarios['Sede'] = df_voluntarios['Sede'].str.replace('Brea','Breña')

### 2.15. Día de voluntariado

In [122]:
df_voluntarios['Día de voluntariado'] = df_voluntarios['Dia de voluntariado']

In [123]:
df_voluntarios.drop(columns=['Dia de voluntariado'], inplace=True)

In [124]:
df_voluntarios['Día de voluntariado'] = df_voluntarios['Día de voluntariado'].str.replace('Mircoles','Miércoles')

In [125]:
df_voluntarios['Día de voluntariado'] = df_voluntarios['Día de voluntariado'].str.replace('Sbado','Sábado')

### 2.16. Turno

In [126]:
df_voluntarios['Turno'] = df_turno[0].str.replace(": //","")

### 2.17. Reglamento voluntariado

In [127]:
L_Reglamento_Antiguo = []
for i in df_voluntarios['Acept acatar el "Reglamento del Voluntariado" de Aprendo Contigo']:
    if i is not None:
        L_Reglamento_Antiguo.append("Sí")
    else:
        L_Reglamento_Antiguo.append("No")
#L_Reglamento_Antiguo

In [128]:
df_voluntarios['Acept acatar el "Reglamento del Voluntariado" de Aprendo Contigo'] = L_Reglamento_Antiguo

In [129]:
L_Reglamento = []
acepto_reg   = df_voluntarios['Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo'].str[:2] == 'on'

for j in acepto_reg:
    if j:
        L_Reglamento.append("Sí")
    else:
        L_Reglamento.append("No")
#L_Reglamento        

In [130]:
df_voluntarios['Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo'] = L_Reglamento

In [131]:
df_voluntarios['Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo'] = df_voluntarios['Acept acatar el "Reglamento del Voluntariado" de Aprendo Contigo'].fillna(df_voluntarios['Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo'])

In [132]:
df_voluntarios['Reglamento Voluntariado'] = df_voluntarios['Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo']

In [133]:
df_voluntarios.drop(columns=['Acepto acatar el Reglamento del Voluntariado de Aprendo Contigo','Acept acatar el "Reglamento del Voluntariado" de Aprendo Contigo'], inplace=True)

### 2.18. Información del programa

In [136]:
L_Info_Programa_Antiguo = []
for i in df_voluntarios['Desea recibir informacin del programa']:
    if i is not None:
        L_Info_Programa_Antiguo.append("Sí")
    else:
        L_Info_Programa_Antiguo.append("No")
#L_Info_Programa_Antiguo

In [137]:
df_voluntarios['Desea recibir informacin del programa'] = L_Info_Programa_Antiguo

In [138]:
L_Info_Programa  = []

acepto_info_prog = df_voluntarios['Deseo recibir informacin del programa'].str[:2] == 'Si'

for j in acepto_info_prog:
    if j:
        L_Info_Programa.append("Sí")
    else:
        L_Info_Programa.append("No")
#L_Info_Programa        

In [139]:
df_voluntarios['Deseo recibir informacin del programa'] = L_Info_Programa

In [140]:
df_voluntarios['Desea recibir informacin del programa'] = df_voluntarios['Deseo recibir informacin del programa'].fillna(df_voluntarios['Desea recibir informacin del programa'])

In [141]:
df_voluntarios['Información del programa'] = df_voluntarios['Desea recibir informacin del programa']

In [142]:
df_voluntarios.drop(columns=['Desea recibir informacin del programa','Deseo recibir informacin del programa'], inplace=True)

### 2.19. Fecha

In [143]:
df_voluntarios['Fecha_Limpia'] = df_voluntarios['Fecha'].str.replace(' a las','')

In [144]:
df_voluntarios['Fecha_Limpia'] = df_voluntarios['Fecha_Limpia'].str.replace(',','')

In [145]:
meses = {
    'enero': '01', 'Jan': '01',
    'febrero': '02', 'Feb': '02',
    'marzo': '03', 'Mar': '03',
    'abril': '04', 'Apr': '04',
    'mayo': '05', 'May': '05',
    'junio': '06', 'Jun': '06',
    'julio': '07', 'Jul': '07',
    'agosto': '08', 'Aug': '08',
    'septiembre': '09', 'setiembre': '09', 'Sep': '09',
    'octubre': '10', 'Oct': '10',
    'noviembre': '11', 'Nov': '11',
    'diciembre': '12', 'Dec': '12'
}

In [146]:
def convertir_fecha_manual(fecha):
    try:
        if pd.isnull(fecha):
            return pd.NaT
        
        partes = fecha.split()
        #print('partes:',partes)
        #print('len:',len(partes))
        
        if len(partes) == 5 and 'de' in partes:  # Formato "día de mes de año"
            dia = partes[0]
            mes = meses.get(partes[2].lower(), None)
            año = partes[4]
        elif len(partes) == 3 and ',' in partes[1]:  # Formato "mes día, año"
            mes = meses.get(partes[0].lower(), None)
            dia = partes[1].replace(',', '')
            año = partes[2]
        elif len(partes) == 3 and 'de' not in partes:  # Otro posible formato "mes día año"
            mes = meses.get(partes[0].lower(), None)
            dia = partes[1]
            año = partes[2]
        elif len(partes) == 4:  # Formato largo en inglés
            dia = partes[1]
            mes = meses.get(partes[2], None)
            año = partes[3]
        else:
            return pd.NaT
        
        if mes is None:  # Si no se encontró el mes en el diccionario
            return pd.NaT
        
        return f"{dia}-{mes}-{año}"
    except Exception as e:
        print(f"Error procesando fecha: {fecha}, Error: {e}")
        return pd.NaT

In [147]:
df_voluntarios['Fecha_Limpia'] = df_voluntarios['Fecha_Limpia'].apply(convertir_fecha_manual)

In [148]:
df_voluntarios['Fecha_Limpia'] = pd.to_datetime(df_voluntarios['Fecha_Limpia'], format='%d-%m-%Y')

In [149]:
df_voluntarios['Fecha'] = df_voluntarios['Fecha_Limpia']

In [150]:
df_voluntarios.drop(columns=['Fecha_Limpia'], inplace=True)

### 2.20. Clasificación

In [151]:
df_voluntarios['Clasificacion'] = np.where(
    (df_voluntarios['Universidad'].notna() & df_voluntarios['Carrera'].notna() & df_voluntarios['Ciclo'].notna()),
    'Estudiante',
    np.where(
        (df_voluntarios['Profesión'].notna() & df_voluntarios['Ocupación'].notna()),
        'Permanente',
        None
    )
)

### 2.21. Timestamp

In [152]:
timestamp_actual = datetime.now()
df_voluntarios['Timestamp extracción'] = timestamp_actual

In [153]:
df_voluntarios

,Nombres y Apellidos,Telfono,Email,Edad,Direccin,Distrito de residencia,Universidad,Carrera,Ciclo,Hospital,...,Turno,Fecha,Motivo,Profesión,Ocupación,Día de voluntariado,Reglamento Voluntariado,Información del programa,Clasificacion,Timestamp extracción
0,Emily Soto Henostroza,929781199,emilyvivianasoto@gmail.com,20,NaN,Comas,Universidad Privada Del Norte,Psicologa,5.0,NaN,...,Mañana: 8:30 am a 1:00pm,2024-07-16,None,NaN,NaN,Lunes Miércoles Viernes,No,Sí,Estudiante,2024-07-16 16:38:54.977152
1,Emily Soto Henostroza,929781199,emilyvivianasoto@gmail.com,20,NaN,Comas,Universidad Privada Del Norte,Psicologa,5.0,NaN,...,Mañana: 8:30 am a 1:00pm,2024-07-16,None,NaN,NaN,Lunes Viernes Miércoles,No,Sí,Estudiante,2024-07-16 16:38:54.977152
2,Emily Soto Henostroza,929781199,emilyvivianasoto@gmail.com,20,NaN,Comas,Universidad Privada Del Norte,Psicologa,5.0,NaN,...,Mañana: 8:30 am a 1:00pm,2024-07-16,None,NaN,NaN,Lunes Miércoles Viernes,No,Sí,Estudiante,2024-07-16 16:38:54.977152
3,Shalom Milagro Velsquez Daz,936858793,1412shalom.vd10@gmail.com,20,NaN,Santa Anita,Universidad Femenina Del Sagrado Corazn,Psicologa,7.0,NaN,...,Tarde: 2:30 pm a 6:00pm,2024-07-16,None,NaN,NaN,Martes Jueves,No,Sí,Estudiante,2024-07-16 16:38:54.977152
4,Abraham Elas Aguilar Hilario,928517058,abrahamhoppers@gmail.com,29,NaN,San Juan De Lurigancho,Universidad Privada Del Norte,Psicologa,10.0,NaN,...,Mañana: 8:30 am a 1:00pm,2024-07-16,None,NaN,NaN,Lunes Martes Miércoles Viernes Sábado,No,Sí,Estudiante,2024-07-16 16:38:54.977152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,Magali Garca Cano,5536585947,maga.garciacano31@gmail.com,43,Ecatepec,None,None,None,NaN,Hospital de Emergencias Peditricas,...,Mañana,2020-03-22,Dese apoyar Ayudar aprender,Estudiante de psicologa,Estudiante de psicologa,None,Sí,No,Permanente,2024-07-16 16:38:54.977152
872,Priscila Mozombite Navarro,934493591,priscilamozombite2019@gmail.com,30,Putumayo 21 calle las mercedes# 26,None,None,None,NaN,Hospital de Emergencias Peditricas,...,Mañana,2020-03-18,Porque me gusta ayudar a las personas me gusta...,Coach finnes,Secundaria completa,None,Sí,No,Permanente,2024-07-16 16:38:54.977152
873,Giancarlo Reyes Aricoche,931099733,gianreyes159707@gmail.com,22,Av.Republica de Panama 4212 Surquillo,None,None,None,NaN,Hospital de Emergencias Peditricas,...,Tarde,2020-03-17,Porque quiero ayudar a las personas que necesi...,Teleoperador en ventas,Teleoperador en ventas,None,Sí,No,Permanente,2024-07-16 16:38:54.977152
874,Mirella Bellido Valentin,969465733,mirellavallinto@gmail.com,22,Mz H1 Lt 11 AV.Miguel Grau Chorrilllos,None,None,None,NaN,Hospital de Emergencias Peditricas,...,Tarde,2020-03-16,Por que quiero ayudar a las personas ser fel...,Arquitectura de Interiores,Estudiante Universitario,None,Sí,No,Permanente,2024-07-16 16:38:54.977152


# III. Exportar a Excel

In [154]:
base_datos_voluntarios = 'C:\\Users\\Daniela\\Aprendo Contigo\\Base de datos - Voluntarios.xlsx'

In [160]:
try:
    existing_data = pd.read_excel(base_datos_voluntarios, sheet_name='Voluntarios')
    primera_fila_vacia = existing_data.shape[0] + 2  # Comienza después de la última fila existente + 1
except FileNotFoundError:
    primera_fila_vacia = 1  # Comienza desde la primera fila si el archivo no existe

# Exportar el DataFrame al archivo Excel existente
with pd.ExcelWriter(base_datos_voluntarios, engine='openpyxl', mode='a') as writer:
    df_voluntarios.to_excel(writer, sheet_name='Base de Datos - Voluntarios', startrow=primera_fila_vacia-1, index=False, header=primera_fila_vacia == 1)

print(f"Datos exportados correctamente a {base_datos_voluntarios} a partir de la fila {primera_fila_vacia}.")

Datos exportados correctamente a C:\Users\Daniela\Aprendo Contigo\Base de datos - Voluntarios.xlsx a partir de la fila 794.
